## Imports

In [1]:
from hops import featurestore

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
41,application_1541520526880_0073,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


## Get Project Featurestore

Each project with the featurestore enabled gets its own Hive database for the featurestore, the name of the featurestore database is 'projectname_featurestore' and can be retrieved from the hops-util-py featurestore API

In [2]:
featurestore.project_featurestore()

'hopsworksdemo_featurestore'

## Get Individual Feature

When retrieving a single feature from the featurestore, the hops-util-py library will infer which featuregroup the feature belongs to by querying the metastore, but you can also explicitly specify which featuregroup and version to query. If there are multiple features of the same name in the featurestore, it is required to specify enough information to uniquely identify the feature (e.g which featuregroup and which version).  If no featurestore is provided it will default to the project's featurestore.

Without specifying featuregroup:

In [3]:
featurestore.get_feature("action").show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT action FROM web_logs_features_1
+------+
|action|
+------+
|     0|
|     0|
|     0|
|     0|
|     0|
+------+
only showing top 5 rows

With specifed featuregroup:

In [4]:
featurestore.get_feature("action", 
                         featurestore=featurestore.project_featurestore(), 
                         featuregroup="web_logs_features", 
                         featuregroup_version = 1).show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT action FROM web_logs_features_1
+------+
|action|
+------+
|     0|
|     0|
|     0|
|     0|
|     0|
+------+
only showing top 5 rows

## Get Featuregroup

You can get an entire featuregroup from the API. If no featurestore is provided the API will default to the project's featurestore, if no version is provided it will default to version 1 of the featuregroup.

In [5]:
featurestore.get_featuregroup("trx_summary_features").show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT * FROM trx_summary_features_1
+-------+---------+---------+---------+---------+
|cust_id|  min_trx|  max_trx|  avg_trx|count_trx|
+-------+---------+---------+---------+---------+
|    148| 390.4109|2094.9958| 1090.509|       16|
|    496| 9.235389|1464.5397| 738.1404|       16|
|    463|33.797318|1828.2426|899.89594|       30|
|    471|578.16833|636.18713|607.17773|        4|
|    243|119.73669| 1582.427| 698.5791|       28|
+-------+---------+---------+---------+---------+
only showing top 5 rows

In [6]:
featurestore.get_featuregroup("trx_summary_features", 
                              featurestore=featurestore.project_featurestore(), 
                              featuregroup_version = 1).show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT * FROM trx_summary_features_1
+-------+---------+---------+---------+---------+
|cust_id|  min_trx|  max_trx|  avg_trx|count_trx|
+-------+---------+---------+---------+---------+
|    148| 390.4109|2094.9958| 1090.509|       16|
|    496| 9.235389|1464.5397| 738.1404|       16|
|    463|33.797318|1828.2426|899.89594|       30|
|    471|578.16833|636.18713|607.17773|        4|
|    243|119.73669| 1582.427| 698.5791|       28|
+-------+---------+---------+---------+---------+
only showing top 5 rows

## Get Set of Features

When retrieving a list of features from the featurestore, the hops-util-py library will infer which featuregroup the features belongs to by querying the metastore. If the features reside in different featuregroups, the library will also **try** to infer how to join the features together based on common columns. If the JOIN query cannot be inferred due to existence of multiple features with the same name or non-obvious JOIN query, the user need to supply enough information to the API call to be able to query the featurestore. If the user already knows the JOIN query it can also run `featurestore.query(joinQuery)` directly. If no featurestore is provided it will default to the project's featurestore.

Without specifying featuregroups and join key:

In [7]:
featurestore.get_features(["pagerank", "triangle_count", "avg_trx"], 
             featurestore=featurestore.project_featurestore()).show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT pagerank, avg_trx, triangle_count FROM trx_graph_summary_features_1 JOIN trx_summary_features_1 ON trx_graph_summary_features_1.`cust_id`=trx_summary_features_1.`cust_id`
+--------+---------+--------------+
|pagerank|  avg_trx|triangle_count|
+--------+---------+--------------+
|     1.0|963.64233|           3.0|
|     1.0| 746.5783|          12.0|
|     1.0|687.91376|           7.0|
|     1.0| 732.6695|          12.0|
|     1.0|  641.785|           4.0|
+--------+---------+--------------+
only showing top 5 rows

Without specifying the join key but specifying featuregroups:

In [8]:
featurestore.get_features(["pagerank", "triangle_count", "avg_trx"], 
             featurestore=featurestore.project_featurestore(), 
             featuregroups_version_dict={
                 "trx_graph_summary_features": 1, 
                "trx_summary_features": 1
             }).show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT pagerank, avg_trx, triangle_count FROM trx_graph_summary_features_1 JOIN trx_summary_features_1 ON trx_graph_summary_features_1.`cust_id`=trx_summary_features_1.`cust_id`
+--------+---------+--------------+
|pagerank|  avg_trx|triangle_count|
+--------+---------+--------------+
|     1.0|963.64233|           3.0|
|     1.0| 746.5783|          12.0|
|     1.0|687.91376|           7.0|
|     1.0| 732.6695|          12.0|
|     1.0|  641.785|           4.0|
+--------+---------+--------------+
only showing top 5 rows

Specifying both featuregroups and join key:

In [9]:
featurestore.get_features(["pagerank", "triangle_count", "avg_trx"], 
             featurestore=featurestore.project_featurestore(), 
             featuregroups_version_dict={
                 "trx_graph_summary_features": 1, 
                "trx_summary_features": 1
             }, 
             join_key="cust_id").show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT pagerank, avg_trx, triangle_count FROM trx_graph_summary_features_1 JOIN trx_summary_features_1 ON trx_graph_summary_features_1.`cust_id`=trx_summary_features_1.`cust_id`
+--------+---------+--------------+
|pagerank|  avg_trx|triangle_count|
+--------+---------+--------------+
|     1.0|963.64233|           3.0|
|     1.0| 746.5783|          12.0|
|     1.0|687.91376|           7.0|
|     1.0| 732.6695|          12.0|
|     1.0|  641.785|           4.0|
+--------+---------+--------------+
only showing top 5 rows

### Advanced examples

Getting 10 features from two different featuregroups without specifying the featuregroups

In [10]:
featurestore.get_features(
    ["pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx",
    "balance", "birthdate", "join_date", "number_of_accounts"], 
             featurestore=featurestore.project_featurestore()).show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT pagerank, count_trx, avg_trx, min_trx, join_date, triangle_count, number_of_accounts, max_trx, balance, birthdate FROM trx_graph_summary_features_1 JOIN trx_summary_features_1 JOIN demographic_features_1 ON trx_graph_summary_features_1.`cust_id`=trx_summary_features_1.`cust_id` AND trx_graph_summary_features_1.`cust_id`=demographic_features_1.`cust_id`
+--------+---------+---------+---------+-------------------+--------------+------------------+---------+---------+-------------------+
|pagerank|count_trx|  avg_trx|  min_trx|          join_date|triangle_count|number_of_accounts|  max_trx|  balance|          birthdate|
+--------+---------+---------+---------+-------------------+--------------+------------------+---------+---------+-------------------+
|     1.0|       16| 1090.509| 390.4109|1998-09-06 00:00:00|           5.0|                10|2094.9958|12920.496|2003-04-12 00:00:00|
|     1.0|       16| 1090.509| 390.4109|1

If you try to get features that exist in multiple featuregroups, the library will not be able to infer from which featuregroup to get the features, so you must specify the featuregroups explicitly as an argument

In [11]:
featurestore.get_features(
    ["pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx",
    "balance", "birthdate", "join_date", "number_of_accounts", "pep"], 
             featurestore=featurestore.project_featurestore()).show(5)

Found the feature with name 'pep' in more than one of the featuregroups of the featurestore: 'hopsworksdemo_featurestore', please specify the optional argument 'featuregroup=', the matched featuregroups were: pep_lookup_1,demographic_features_1
Traceback (most recent call last):
  File "/srv/hops/anaconda/anaconda/envs/hopsworksdemo/lib/python3.6/site-packages/hops/featurestore.py", line 325, in get_features
    featuregroup_matched = _find_feature(feature, featurestore, featuregroups_parsed)
  File "/srv/hops/anaconda/anaconda/envs/hopsworksdemo/lib/python3.6/site-packages/hops/featurestore.py", line 171, in _find_feature
    featuregroups_matched_str))
AssertionError: Found the feature with name 'pep' in more than one of the featuregroups of the featurestore: 'hopsworksdemo_featurestore', please specify the optional argument 'featuregroup=', the matched featuregroups were: pep_lookup_1,demographic_features_1



If we specify the featuregroup to get the feature where that exists in multiple featuregroups, the library can infer how to get the features:

In [12]:
featurestore.get_features(
    ["pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx",
    "balance", "birthdate", "join_date", "number_of_accounts", "pep"], 
             featurestore=featurestore.project_featurestore(),
    featuregroups_version_dict={
                "trx_graph_summary_features": 1, 
                "trx_summary_features": 1,
                "demographic_features": 1
             }).show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT pagerank, count_trx, avg_trx, min_trx, join_date, triangle_count, number_of_accounts, max_trx, pep, balance, birthdate FROM demographic_features_1 JOIN trx_graph_summary_features_1 JOIN trx_summary_features_1 ON demographic_features_1.`cust_id`=trx_graph_summary_features_1.`cust_id` AND demographic_features_1.`cust_id`=trx_summary_features_1.`cust_id`
+--------+---------+---------+---------+-------------------+--------------+------------------+---------+-------------+---------+-------------------+
|pagerank|count_trx|  avg_trx|  min_trx|          join_date|triangle_count|number_of_accounts|  max_trx|          pep|  balance|          birthdate|
+--------+---------+---------+---------+-------------------+--------------+------------------+---------+-------------+---------+-------------------+
|     1.0|       16|963.64233|166.16486|1994-11-18 00:00:00|           3.0|                11|1853.4875|1331439861760|13078.942|1987-06

Example of getting 19 features from 5 different featuregroups: 

In [13]:
featurestore.get_features(
    ["pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx",
    "balance", "birthdate", "join_date", "number_of_accounts", "pep", "customer_type", "gender", "web_id",
    "time_spent_seconds", "address", "action", "report_date", "report_id"], 
             featurestore=featurestore.project_featurestore(),
    featuregroups_version_dict={
                "trx_graph_summary_features": 1, 
                "trx_summary_features": 1,
                "demographic_features": 1,
                "web_logs_features": 1,
                "police_report_features": 1
             }).show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT pagerank, report_date, report_id, gender, count_trx, avg_trx, customer_type, address, time_spent_seconds, web_id, min_trx, join_date, triangle_count, number_of_accounts, action, max_trx, pep, balance, birthdate FROM demographic_features_1 JOIN trx_graph_summary_features_1 JOIN trx_summary_features_1 JOIN police_report_features_1 JOIN web_logs_features_1 ON demographic_features_1.`cust_id`=trx_graph_summary_features_1.`cust_id` AND demographic_features_1.`cust_id`=trx_summary_features_1.`cust_id` AND demographic_features_1.`cust_id`=police_report_features_1.`cust_id` AND demographic_features_1.`cust_id`=web_logs_features_1.`cust_id`
+--------+-------------------+---------+------------+---------+--------+-------------+-------+------------------+------+--------+-------------------+--------------+------------------+------+---------+-------------+--------+-------------------+
|pagerank|        report_date|report_id|      gender

Sometimes you might want to get a feature that exist in multiple featuregroups and you want to include all of these featuregroups in your query, then you can specify from which of the featuregroup to get the feature by prepending the feature-name with the featuregroup name + '\_version', e.g: 'demographic\_features_1.cust_id'. If you don't specify this the query will fail as the library won't know from which of your specified featuregroups to get the feature: 

In [14]:
featurestore.get_features(
    ["pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx",
    "balance", "birthdate", "join_date", "number_of_accounts", "pep", "customer_type", "gender", "web_id",
    "time_spent_seconds", "address", "action", "report_date", "report_id", "cust_id"], 
             featurestore=featurestore.project_featurestore(),
    featuregroups_version_dict={
                "trx_graph_summary_features": 1, 
                "trx_summary_features": 1,
                "demographic_features": 1,
                "web_logs_features": 1,
                "police_report_features": 1
             }).show(5)

"Reference 'cust_id' is ambiguous, could be: demographic_features_1.cust_id, trx_graph_summary_features_1.cust_id, trx_summary_features_1.cust_id, police_report_features_1.cust_id, web_logs_features_1.cust_id.; line 1 pos 219"
Traceback (most recent call last):
  File "/srv/hops/anaconda/anaconda/envs/hopsworksdemo/lib/python3.6/site-packages/hops/featurestore.py", line 315, in get_features
    return _run_and_log_sql(spark, sql_str)
  File "/srv/hops/anaconda/anaconda/envs/hopsworksdemo/lib/python3.6/site-packages/hops/featurestore.py", line 29, in _run_and_log_sql
    return spark.sql(sql_str)
  File "/srv/hops/hopsdata/tmp/nm-local-dir/usercache/wmHx4l6IX6DR5AHnPc6B-vkoG-kzfSsDlfaLhxExZn0/appcache/application_1541520526880_0073/container_e01_1541520526880_0073_01_000001/pyspark.zip/pyspark/sql/session.py", line 710, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/srv/hops/hopsdata/tmp/nm-local-dir/usercache/wmHx4l6IX6DR5AHnPc6B-vkoG-kzfSsDlfaLh

If we change 'cust\_id' to 'featuregroupname\_version.cust\_id' it works: 

In [15]:
featurestore.get_features(
    ["pagerank", "triangle_count", "avg_trx", "count_trx", "max_trx", "min_trx",
    "balance", "birthdate", "join_date", "number_of_accounts", "pep", "customer_type", "gender", "web_id",
    "time_spent_seconds", "address", "action", "report_date", "report_id", "demographic_features_1.cust_id"], 
             featurestore=featurestore.project_featurestore(),
    featuregroups_version_dict={
                "trx_graph_summary_features": 1, 
                "trx_summary_features": 1,
                "demographic_features": 1,
                "web_logs_features": 1,
                "police_report_features": 1
             }).show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT pagerank, report_date, report_id, gender, demographic_features_1.cust_id, count_trx, avg_trx, customer_type, address, time_spent_seconds, web_id, min_trx, join_date, triangle_count, number_of_accounts, action, max_trx, pep, balance, birthdate FROM demographic_features_1 JOIN trx_graph_summary_features_1 JOIN trx_summary_features_1 JOIN police_report_features_1 JOIN web_logs_features_1 ON demographic_features_1.`cust_id`=trx_graph_summary_features_1.`cust_id` AND demographic_features_1.`cust_id`=trx_summary_features_1.`cust_id` AND demographic_features_1.`cust_id`=police_report_features_1.`cust_id` AND demographic_features_1.`cust_id`=web_logs_features_1.`cust_id`
+--------+-------------------+---------+------------+-------+---------+--------+-------------+-------+------------------+------+--------+-------------------+--------------+------------------+------+---------+-------------+--------+-------------------+
|pagerank|  

## Free Query Feature Store

For complex queries that cannot be inferred by the helper functions, enter the sql directly to the method `featurestore.sql()` it will default to the project specific feature store but you can also specify it explicitly.

Without specifying the featurestore it will default to the project-specific featurestore:

In [16]:
featurestore.sql("SELECT * FROM trx_graph_summary_features_1 WHERE triangle_count > 5").show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT * FROM trx_graph_summary_features_1 WHERE triangle_count > 5
+-------+--------+--------------+
|cust_id|pagerank|triangle_count|
+-------+--------+--------------+
|     29|     1.0|          12.0|
|    474|     1.0|           7.0|
|     65|     1.0|          12.0|
|    222|     1.0|          13.0|
|    270|     1.0|           8.0|
+-------+--------+--------------+
only showing top 5 rows

You can also specify the featurestore to query explicitly:

In [17]:
featurestore.sql("SELECT * FROM trx_graph_summary_features_1 WHERE triangle_count > 5", 
                 featurestore=featurestore.project_featurestore()).show(5)

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT * FROM trx_graph_summary_features_1 WHERE triangle_count > 5
+-------+--------+--------------+
|cust_id|pagerank|triangle_count|
+-------+--------+--------------+
|     29|     1.0|          12.0|
|    474|     1.0|           7.0|
|     65|     1.0|          12.0|
|    222|     1.0|          13.0|
|    270|     1.0|           8.0|
+-------+--------+--------------+
only showing top 5 rows

## Write to the Feature Store

Lets first get some sample data to insert

In [18]:
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType
sqlContext = SQLContext(sc)
schema = StructType([StructField("id", LongType(), True),
                         StructField("customer_type", StringType(), True)
                        ])
sampleDf = sqlContext.createDataFrame([(3, "hops_customer_1"), (4, "hops_customer_2")], schema)

In [19]:
sampleDf.show()

+---+---------------+
| id|  customer_type|
+---+---------------+
|  3|hops_customer_1|
|  4|hops_customer_2|
+---+---------------+

Lets inspect the contents of the featuregroup 'customer_type_lookup' that we are going to insert the sample data into

In [20]:
sparkDf = featurestore.get_featuregroup("customer_type_lookup")

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT * FROM customer_type_lookup_1

In [21]:
sparkDf.show()

+-------------+------------+
|customer_type|          id|
+-------------+------------+
|    corporate|420906795008|
|      private|893353197568|
+-------------+------------+

In [22]:
sparkDf.count()

2

Now we can insert the sample data and verify the new contents of the featuregroup. By default the insert mode is "append", the featurestore is the project's featurestore and the version is 1

In [23]:
featurestore.insert_into_featuregroup(sampleDf, "customer_type_lookup")

Running sql: use hopsworksdemo_featurestore

In [24]:
featurestore.get_featuregroup("customer_type_lookup").show()

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT * FROM customer_type_lookup_1
+---------------+------------+
|  customer_type|          id|
+---------------+------------+
|hops_customer_1|           3|
|hops_customer_2|           4|
|      corporate|420906795008|
|        private|893353197568|
+---------------+------------+

In [25]:
featurestore.get_featuregroup("customer_type_lookup").count()

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT * FROM customer_type_lookup_1
4

You can also explicitly specify featurestore, featuregroup version, and the insert mode:

In [26]:
featurestore.insert_into_featuregroup(sampleDf, 
                         "customer_type_lookup", 
                         featurestore=featurestore.project_featurestore(), 
                         featuregroup_version=1, 
                         mode="append")

Running sql: use hopsworksdemo_featurestore

In [27]:
featurestore.get_featuregroup("customer_type_lookup").show()

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT * FROM customer_type_lookup_1
+---------------+------------+
|  customer_type|          id|
+---------------+------------+
|hops_customer_1|           3|
|hops_customer_1|           3|
|hops_customer_2|           4|
|hops_customer_2|           4|
|      corporate|420906795008|
|        private|893353197568|
+---------------+------------+

The two supported insert modes are "append" and "overwrite"

In [28]:
featurestore.insert_into_featuregroup(sampleDf, 
                         "customer_type_lookup", 
                         featurestore=featurestore.project_featurestore(), 
                         featuregroup_version=1, 
                         mode="overwrite")

Running sql: use hopsworksdemo_featurestore

In [29]:
featurestore.get_featuregroup("customer_type_lookup").show()

Running sql: use hopsworksdemo_featurestore
Running sql: SELECT * FROM customer_type_lookup_1
+---------------+---+
|  customer_type| id|
+---------------+---+
|hops_customer_1|  3|
|hops_customer_2|  4|
+---------------+---+

## Get Featurestore Metadata
To explore the contents of the featurestore we recommend using the featurestore page in the Hopsworks UI but you can also get the metadata programmatically from the REST API with the following method

In [30]:
featurestore.get_featurestore_metadata()

[{'featuregroupId': 46, 'featuregroupName': 'customer_type_lookup', 'featurestoreId': 1, 'featurestoreName': 'hopsworksdemo_featurestore', 'description': 'lookup table for id to customer type, used when converting from numeric to categrorical representation and vice verse', 'inputDataset': 'hdfs:///Projects/hopsworksdemo/sample_data/kyc.csv', 'hdfsStorePaths': ['hdfs://10.0.2.15:8020/apps/hive/warehouse/hopsworksdemo_featurestore.db/customer_type_lookup_1'], 'created': '2018-11-08-09:55:23', 'creator': 'admin@kth.se', 'features': [{'name': 'customer_type', 'type': 'string', 'description': 'The categorical customer_type', 'primary': False}, {'name': 'id', 'type': 'bigint', 'description': 'The numeric id of the customer_type', 'primary': True}], 'jobId': 20, 'jobName': 'customer_type_lookup_job', 'lastComputed': '2018-11-08-09:51:10', 'jobStatus': 'Succeeded', 'version': 1}, {'featuregroupId': 21, 'featuregroupName': 'pep_lookup', 'featurestoreId': 1, 'featurestoreName': 'hopsworksdemo_f

In [31]:
featurestore.get_featurestore_metadata(featurestore=featurestore.project_featurestore())

[{'featuregroupId': 46, 'featuregroupName': 'customer_type_lookup', 'featurestoreId': 1, 'featurestoreName': 'hopsworksdemo_featurestore', 'description': 'lookup table for id to customer type, used when converting from numeric to categrorical representation and vice verse', 'inputDataset': 'hdfs:///Projects/hopsworksdemo/sample_data/kyc.csv', 'hdfsStorePaths': ['hdfs://10.0.2.15:8020/apps/hive/warehouse/hopsworksdemo_featurestore.db/customer_type_lookup_1'], 'created': '2018-11-08-09:55:23', 'creator': 'admin@kth.se', 'features': [{'name': 'customer_type', 'type': 'string', 'description': 'The categorical customer_type', 'primary': False}, {'name': 'id', 'type': 'bigint', 'description': 'The numeric id of the customer_type', 'primary': True}], 'jobId': 20, 'jobName': 'customer_type_lookup_job', 'lastComputed': '2018-11-08-09:51:10', 'jobStatus': 'Succeeded', 'version': 1}, {'featuregroupId': 21, 'featuregroupName': 'pep_lookup', 'featurestoreId': 1, 'featurestoreName': 'hopsworksdemo_f